In [1]:
import matplotlib.pyplot as plt 
%matplotlib inline
import numpy as np 
np.set_printoptions(precision=4)

In [2]:
import numpy as np 
import threading
import os
import glob 
from skimage.io import imread
from random import shuffle
import cv2


class Generator:  
    
    def __init__(self, directory, batch_size, grayscale=False, normalize=True, SVD=False, flip=False):
        
        self.directory = directory
        self.batch_size = batch_size
        self.img_dir = None
        self.low_rank_imgs = None
        self.lock = threading.Lock()
        
        #setup data
        self.setup_data()          
        self.setup_lowranks()
        
        #for frame shift of images in list img_dir
        self.start = 0
        self.end = self.batch_size 
        
        #init output batch
        if grayscale:
            self.X_batch = np.zeros((batch_size, 180, 320, 1))
        else:  
            self.X_batch = np.zeros((batch_size, 180, 320, 3))
        self.y_batch = np.zeros((batch_size, 1))
        
        #augment 
        self.SVD = SVD
        self.flip = flip 
        self.normalize = normalize
        self.grayscale = grayscale

    def setup_data(self):
        '''get and verify images are in directory, labels are correct '''
        
        self.img_dir = glob.glob(self.directory + '/images/*')
        shuffle(self.img_dir)
        self.num_batches = int(np.ceil(len(self.img_dir)/ self.batch_size)) #number of batches 

        assert len(self.img_dir) > 0, 'no images found, check directory'

    def setup_lowranks(self):
        '''setup the preprocessed SVD lowrank images
        build a dict of names : image tensors'''
        
        self.low_rank_imgs = {}
        img_list = glob.glob(self.directory + '/*low_rank.png')
        
        for im in img_list: 
            #same name formatting 
            name = im.replace(self.directory, '')
            name = name.replace('\\', '')
            name = name.split('_')[:2]
            name = '{}_{}'.format(name[0], name[1])  
            self.low_rank_imgs[name] = imread(im)
    
    def get_img_metadata(self, img_name):
        '''get metadata from image name 
        returns a dict with keys: 
        
        name : (str) video name
        frame : (int) frame number 
        label : (float) steering angle
        array : image tensor'''
        
        unique_name = img_name.split('epoch')[-1].split('_')
        label = unique_name[-1].split('.')[-2]
        vid_name = 'epoch' + unique_name[0] + '_' + unique_name[1]
        
        
        d = {}
        d['label'] = float(label)/10
        d['frame'] = int(unique_name[-2])
        d['name'] = vid_name
        d['array'] = imread(img_name) #self.rgb2gray()
        
        return d
    
    def remove_background_SVD(self, img_data):
        '''get low rank image from frame's parent video 
        subtract from image array, in place for dict. 
        Apply weighed average of frame and low-rank matrix
        '''
        low_rank = self.low_rank_imgs[img_data['name']]
        diff = 255- cv2.absdiff(img_data['array'] , low_rank)
        img_data['array']  = cv2.addWeighted(diff, 0.9, img_data['array'], 0.1, 1)

        
    def flip_img(self, img_data):
        '''randomly flip the image and reverse the steering angle
        50/50 odds'''
        if np.random.randint(0, 2): 
            img_data['array'] = np.flip(img_data['array'], axis=1)
            img_data['label'] *= -1 
            
    def normalize_img(self, img_data): 
        '''normalize the image tensor [-1, 1]'''
        mini = np.min(img_data['array'])
        maxi = np.max(img_data['array'])
        img_data['array'] = img_data['array']/255
        #img_data['array'] = 2*((img_data['array'] - mini) / (maxi-mini))-1
    
    def rgb2gray(self, rgb):
        '''cvt image from RGB to grayscale'''
        return np.dot(rgb[:, :, :3], [0.299, 0.587, 0.114])

    def __next__(self): 
        '''Yields data tensor of size [batch_size, 180, 320, 1], 
        label tensor of size [batch_size, 1]. GPU compatible. '''

        #lock and release threads at iteration execution 
        with self.lock:      
            for i in range(self.num_batches):
                img_batch_files = self.img_dir[self.start:self.end]

                for j, img_name in enumerate(img_batch_files): 
                    img_data = self.get_img_metadata(img_name) #image data dict
                     
                    #augment 
                    if self.SVD: 
                        self.remove_background_SVD(img_data)
                    if self.flip:
                        self.flip_img(img_data)
                    if self.normalize:
                        self.normalize_img(img_data)
                    if self.grayscale:
                        self.rgb2gray(img_data)
                    
                    self.X_batch[j, :, :, :] = img_data['array'].reshape(180,320,-1)
                    self.y_batch[j] = img_data['label'] #get steering angle

                #clip last batch 
                if i == self.num_batches - 1:
                    self.X_batch = self.X_batch[:j, :, :, :]       

                #increment images for next iteration 
                self.start += self.batch_size
                self.end += self.batch_size
                
                return  self.X_batch, self.y_batch
                
    def __iter__(self):
        return self

Test is out 

In [3]:
d = os.path.join('..', 'data', 'train')
# d = os.path.join('/input', 'train')

gen = Generator(directory = d, 
                batch_size = 200,
                grayscale=False,
                normalize = True,
                SVD = True,
                flip = False)

In [4]:
gen.img_dir;
# os.listdir(os.path.join('..', 'data', 'train'))

In [5]:
#print out the generator output until the StopIteration error
try: 
    for i in range(10): 
        x_batch, y_batch = next(gen)
        print ('batch {}, size: {}, x_mean: {}, y_mean {}'\
               .format(i, x_batch.shape, np.mean(x_batch), np.mean(y_batch)))
       
        #visualize results!  
#         plt.figure(figsize=(20, 10))
#         plt.title('SVD applied')
#         for i in range(16):
#             plt.subplot(4, 4, i+1)
#             plt.imshow(x_batch[i, :, :, :].reshape(180, 320, -1), cmap='gray')
#         plt.show()

except StopIteration: 
    print ('reached end of image data')

batch 0, size: (200, 180, 320, 3), x_mean: 0.8549380949301028, y_mean 0.1975
batch 1, size: (200, 180, 320, 3), x_mean: 0.8530433072349318, y_mean -0.5425
batch 2, size: (200, 180, 320, 3), x_mean: 0.851051974514342, y_mean 0.395
batch 3, size: (200, 180, 320, 3), x_mean: 0.8553441099310101, y_mean -0.61
batch 4, size: (200, 180, 320, 3), x_mean: 0.855285477146877, y_mean 0.2
batch 5, size: (200, 180, 320, 3), x_mean: 0.8548185861474171, y_mean 0.085
batch 6, size: (200, 180, 320, 3), x_mean: 0.8551497800925922, y_mean -0.135
batch 7, size: (200, 180, 320, 3), x_mean: 0.8556509058415029, y_mean -0.2075
batch 8, size: (200, 180, 320, 3), x_mean: 0.8558581161492363, y_mean -0.34
batch 9, size: (200, 180, 320, 3), x_mean: 0.8550599362291225, y_mean -0.3825


This looks good. I will save it to a .py file and use it for the model. 